# Getting Started

## Introduction
ReSyPE (pronounced *recipe*) is a Python library built for both practitioners and researchers that wish to quickly develop and deploy ML-based recommender systems.

The library provides an end-to-end pipeline that includes:

1. Loading transaction, user feature, and item feature datasets
2. Interchangable methods for user and item clustering
3. Modular framework for machine learning models
4. Iterative and decomposition-based techniques


## Installation

`pip install resype`


## Load the Data

In [1]:
import pandas as pd
import numpy as np
from resype.resype import Resype

%load_ext autoreload
%autoreload 2 

In [2]:
# load transaction list
transaction_list = pd.read_csv("sample_data/ratings.csv")[['userId', 'movieId', 'rating']]
transaction_list = transaction_list.sample(20)
transaction_list.columns = ["user_id", 'item_id', 'rating']

## Preprocess

In [3]:
re = Resype(transaction_list)
re.transaction_list.head(3)

,user_id,item_id,rating
91090,590,6783,3.5
2239,18,122920,3.0
62507,414,481,3.0


In [4]:
# construct utlity matrix
re.construct_utility_matrix()
re.utility_matrix.head(3)

item_id,481,780,783,788,1136,1196,1215,1288,1374,1882,1944,2427,2688,3247,4452,5989,6783,7438,122920,173535
user_id,,,,,,,,,,,,,,,,,,,,
18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN
20,NaN,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Train on Unclustered Matrix

### Iterative Approach

In [5]:
# import sklearn Model
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()

In [6]:
# fit and predict
re.fit(model, method='iterative')
re.utility_matrix_preds.head(3)

item_id,481,780,783,788,1136,1196,1215,1288,1374,1882,1944,2427,2688,3247,4452,5989,6783,7438,122920,173535
user_id,,,,,,,,,,,,,,,,,,,,
18,0.5,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.333333,-0.5,0.000000e+00,0.333333,-0.666667,0.5,-0.5,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
20,0.5,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.333333,-0.5,0.000000e+00,0.333333,-0.666667,0.5,-0.5,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
42,0.5,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,0.333333,-0.5,1.480297e-16,0.333333,-0.666667,0.5,-0.5,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16


In [7]:
# recommend
user_list = [0, 1, 2] # indices
top_n = 10
re.get_rec(user_list, top_n)
re.df_rec

,user_id,rank_1,rank_2,rank_3,rank_4,rank_5,rank_6,rank_7,rank_8,rank_9,rank_10
0,18,481.0,3247.0,1374.0,2427.0,780.0,783.0,788.0,1136.0,1196.0,1215.0
1,20,481.0,3247.0,1374.0,2427.0,780.0,788.0,1136.0,1196.0,1215.0,1288.0
2,42,481.0,3247.0,122920.0,780.0,783.0,788.0,1136.0,1196.0,1215.0,1288.0


## Train on Clustered matrix

In [8]:
from sklearn.cluster import KMeans
km_users = KMeans(n_clusters=10)
km_items = KMeans(n_clusters=10)

user_model, user_cluster_map, util_matrix_w_users = re.cluster_users(km_users)
item_model, item_cluster_map, util_matrix_w_items = re.cluster_items(km_items)

In [9]:
re.cluster_assignment()
re.utility_matrix_agg(u_agg="mean", i_agg="mean")
re.utility_matrix.head(3)

i_cluster,0,1,2,3,4,5,6,7,8,9
u_cluster,,,,,,,,,,
0,0.0,0.0,3.666667,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,4.0,0.0
2,0.5,0.0,0.000000,0.330357,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
from sklearn.ensemble import RandomForestRegressor
model1 = RandomForestRegressor()
re.fit(model_object=model1, method='iterative', n_synth_data=5, p=0.1)
re.utility_matrix_preds.head(3)

i_cluster,0,1,2,3,4,5,6,7,8,9
u_cluster,,,,,,,,,,
0,-0.411481,-0.397742,3.255185,-0.381283,-0.411481,-0.181846,-0.411481,-0.411481,0.171945,-0.411481
1,-0.448889,-0.448889,-0.315947,-0.429701,-0.448889,-0.448889,-0.199500,-0.448889,3.551111,-0.369698
2,0.410967,-0.089033,-0.079824,0.241324,-0.089033,-0.089033,-0.058418,-0.089033,0.151372,-0.089033


In [11]:
# predict top item clusters per user index
user_list = [0, 1, 2] # index
top_n = 5 # top n clusters
re.get_rec(user_list, top_n, re.user_assignment)

,user_id,rank_1,rank_2,rank_3,rank_4,rank_5
0,0,8.0,6.0,2.0,9.0,7.0
1,1,8.0,6.0,2.0,9.0,7.0
2,2,8.0,5.0,3.0,1.0,9.0


In [12]:
# predict top items per user_id
top_n = 5 # top n clusters
re.get_rec_item(top_n)

,user_id,rank_1,rank_2,rank_3,rank_4,rank_5
0,18,1944.0,173535.0,1374.0,2688.0,2427.0
1,20,1944.0,173535.0,2427.0,2688.0,1374.0
2,42,1944.0,1215.0,1288.0,4452.0,481.0
